<a href="https://colab.research.google.com/github/elly-4/aegle/blob/master/Neurological_CP_of_ColabProj0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install and Import dependencies**

In [ ]:
!pip install mediapipe opencv-python


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/comm

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import mediapipe as mp
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import PIL
import io
import html
import time
from google.colab.patches import cv2_imshow


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


**Get Realtime Webcam Feed**

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

## **Xtremely long video capture code**

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label_html, bbox ):
  data = eval_js('stream_frame("{}", "{}")'.format(label_html, bbox))
  return data

## **Calling the video function**

In [ ]:
# start streaming video from webcam
video_stream()

# # label for video
label_html = 'Capturing...' 
# initialze bounding box to empty
bbox = ''
count = 0 
# image = '' 

#Intiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while True:
      js_reply = video_frame(label_html, bbox )
    
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      #Recolour feed
      image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      #Make Detections
      results = holistic.process(image)
      print(results.face_landmarks)

      #face_Landmarks, pose_Landmarks, left_Landmarks, right_Landmarks

      #Recolor image back ot BGR for Rendering 
      image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

      #Draw Face landmarks 
      # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)

      #Right hand 
      mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      #Left hand
      mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      # Pose Detections
      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
     
      cv2_imshow(image)

In [ ]:

results.face_landmarks.landmark[0].visibility

## 2. **Capture Landmarks & Export to CSV** 

In [ ]:
import csv
import os
import numpy as np


In [ ]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

### **landmarks**

In [ ]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [ ]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### **ClassName Set**

In [ ]:

class_name = "HipAbductors3"

In [ ]:
# start streaming video from webcam
video_stream()

# # label for video
label_html = 'Capturing...' 
# initialze bounding box to empty
bbox = ''
count = 0 
# image = '' 

#Intiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while True:
      js_reply = video_frame(label_html, bbox )
    
      if not js_reply:
       break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      #Recolour feed
      image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      #Make Detections
      results = holistic.process(image)
      print(results.face_landmarks)

      #face_Landmarks, pose_Landmarks, left_Landmarks, right_Landmarks

      #Recolor image back ot BGR for Rendering 
      image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

      #Draw Face landmarks 
      # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)

      #Right hand 
      mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      #Left hand
      mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
      # 4. Pose Detections
      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
      # Export coordinates
      try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
      except:
            pass

    
      cv2_imshow(image)
 

## **3. Train Custom Model Using Scikit Learn**

### **3.1 Read in Collected Data and Process**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('coords.csv')

In [ ]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,z3,v3,x4,y4,z4,v4,x5,y5,z5,v5,x6,y6,z6,v6,x7,y7,z7,v7,x8,y8,z8,v8,x9,y9,z9,v9,x10,y10,z10,...,x492,y492,z492,v492,x493,y493,z493,v493,x494,y494,z494,v494,x495,y495,z495,v495,x496,y496,z496,v496,x497,y497,z497,v497,x498,y498,z498,v498,x499,y499,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,FacialAppearanceScore3,0.534910,0.607808,-2.742623,0.998553,0.586397,0.464776,-2.633734,0.998995,0.625719,0.459521,-2.634023,0.998560,0.653113,0.457898,-2.635192,0.998819,0.480927,0.481279,-2.639994,0.999083,0.441230,0.486824,-2.641192,0.998664,0.409270,0.494262,-2.643181,0.998913,0.728609,0.514328,-1.711463,0.998774,0.365171,0.553295,-1.737519,0.998794,0.614693,0.735148,-2.362345,...,0.568488,0.646702,-0.081472,0.0,0.578697,0.637597,-0.080099,0.0,0.598396,0.645646,-0.036692,0.0,0.565076,0.649901,-0.086273,0.0,0.563316,0.653988,-0.063310,0.0,0.592129,0.468725,0.006596,0.0,0.580978,0.476911,-0.005030,0.0,0.574946,0.483255,-0.018556,0.0,0.678364,0.457119,0.017886,0.0,0.687865,0.442695,0.018433,0.0
1,FacialAppearanceScore3,0.539963,0.607300,-2.501284,0.998497,0.592415,0.467861,-2.387819,0.998955,0.631042,0.464062,-2.387945,0.998468,0.659372,0.462473,-2.389120,0.998763,0.484304,0.481979,-2.395126,0.999051,0.444921,0.486932,-2.396187,0.998608,0.411357,0.494082,-2.398216,0.998882,0.730830,0.517257,-1.461478,0.998702,0.366085,0.553239,-1.498547,0.998698,0.617600,0.735193,-2.121241,...,0.572117,0.646056,-0.080379,0.0,0.582014,0.637037,-0.078947,0.0,0.601274,0.646004,-0.036388,0.0,0.568777,0.649114,-0.085114,0.0,0.567007,0.653409,-0.062740,0.0,0.594899,0.470575,0.007390,0.0,0.584136,0.478375,-0.003976,0.0,0.578377,0.484423,-0.017183,0.0,0.680568,0.460181,0.018647,0.0,0.690172,0.445442,0.019310,0.0
2,FacialAppearanceScore3,0.546876,0.606053,-2.544703,0.998410,0.599532,0.470442,-2.439581,0.998919,0.637453,0.468309,-2.439628,0.998299,0.666346,0.467716,-2.440818,0.998688,0.491853,0.482043,-2.448687,0.999029,0.452536,0.486711,-2.450041,0.998527,0.418648,0.492766,-2.452127,0.998874,0.734411,0.521187,-1.529718,0.998546,0.369989,0.552046,-1.572599,0.998578,0.621421,0.735164,-2.171888,...,0.585848,0.640872,-0.079928,0.0,0.595621,0.631616,-0.078450,0.0,0.615024,0.642037,-0.035930,0.0,0.582544,0.643938,-0.084673,0.0,0.580924,0.648550,-0.062382,0.0,0.607275,0.464980,0.007782,0.0,0.596586,0.473255,-0.003630,0.0,0.590963,0.479527,-0.016874,0.0,0.691652,0.454417,0.019110,0.0,0.701079,0.439927,0.019840,0.0
3,FacialAppearanceScore3,0.550979,0.604826,-2.531286,0.998423,0.603472,0.471482,-2.422215,0.998934,0.640735,0.470085,-2.422304,0.998288,0.669820,0.469985,-2.423356,0.998694,0.497182,0.482050,-2.436733,0.999051,0.457970,0.486555,-2.437951,0.998548,0.424029,0.492129,-2.440059,0.998909,0.736055,0.523183,-1.492297,0.998535,0.372770,0.551783,-1.563339,0.998582,0.623601,0.734954,-2.150120,...,0.586047,0.640313,-0.079637,0.0,0.595908,0.631190,-0.078094,0.0,0.614781,0.642057,-0.035766,0.0,0.582854,0.643257,-0.084402,0.0,0.581023,0.647946,-0.062211,0.0,0.607855,0.468151,0.007663,0.0,0.597280,0.475706,-0.003730,0.0,0.591675,0.481414,-0.016948,0.0,0.692197,0.457998,0.019905,0.0,0.701685,0.443267,0.020664,0.0
4,FacialAppearanceScore3,0.557918,0.605315,-2.823862,0.998506,0.609072,0.475497,-2.709193,0.998985,0.645480,0.474816,-2.709430,0.998357,0.675137,0.475241,-2.710208,0.998750,0.503696,0.483541,-2.717139,0.999108,0.464906,0.487117,-2.718207,0.998628,0.430866,0.492288,-2.720174,0.998978,0.739210,0.526507,-1.719085,0.998587,0.378430,0.551786,-1.756949,0.998664,0.627870,0.736315,-2.415689,...,0.590396,0.631863,-0.078843,0.0,0.600136,0.622781,-0.077136,0.0,0.618708,0.631797,-0.035298,0.0,0.587257,0.634849,-0.083552,0.0,0.585456,0.639350,-0.061661,0.0,0.611595,0.462842,0.009679,0.0,0.600909,0.470478,-0.001886,0.0,0.595255,0.476170,-0.015155,0.0,0.694821,0.452397,0.022259,0.0,0.703991,0.438335,0.023126,0.0


In [ ]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,z3,v3,x4,y4,z4,v4,x5,y5,z5,v5,x6,y6,z6,v6,x7,y7,z7,v7,x8,y8,z8,v8,x9,y9,z9,v9,x10,y10,z10,...,x492,y492,z492,v492,x493,y493,z493,v493,x494,y494,z494,v494,x495,y495,z495,v495,x496,y496,z496,v496,x497,y497,z497,v497,x498,y498,z498,v498,x499,y499,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
440,LegsSitting0,0.502616,0.090131,-0.614206,0.999999,0.517278,0.029207,-0.590349,0.999997,0.537900,0.025603,-0.590671,0.999999,0.559748,0.023022,-0.591148,0.999997,0.465375,0.043209,-0.550334,0.999994,0.449941,0.049549,-0.550424,0.999997,0.436256,0.056547,-0.550026,0.999987,0.601406,0.053327,-0.388927,0.999999,0.432878,0.102339,-0.209537,0.999989,0.558791,0.149890,-0.537329,...,0.501821,0.119616,-0.034268,0.0,0.505199,0.114481,-0.033996,0.0,0.516586,0.118442,-0.017569,0.0,0.500396,0.121131,-0.036101,0.0,0.500739,0.123736,-0.027057,0.0,0.507982,0.042525,0.003904,0.0,0.503318,0.046901,-0.000207,0.0,0.500309,0.049809,-0.005252,0.0,0.546412,0.024383,0.004373,0.0,0.549899,0.016568,0.004348,0.0
441,LegsSitting0,0.518970,0.098765,-0.449570,0.999999,0.539361,0.038271,-0.423863,0.999997,0.558073,0.032329,-0.424218,0.999998,0.575525,0.028161,-0.424711,0.999997,0.488190,0.058542,-0.387848,0.999994,0.472817,0.066091,-0.387918,0.999996,0.455869,0.074352,-0.387602,0.999986,0.622299,0.055941,-0.245300,0.999998,0.456574,0.118530,-0.087815,0.999985,0.574472,0.153304,-0.383513,...,0.549585,0.131330,-0.032926,0.0,0.552841,0.126471,-0.032716,0.0,0.564863,0.130167,-0.018318,0.0,0.547987,0.132602,-0.034688,0.0,0.548691,0.135292,-0.026494,0.0,0.559317,0.062013,0.006185,0.0,0.554473,0.065725,0.002437,0.0,0.551090,0.068016,-0.002368,0.0,0.595570,0.045385,0.004978,0.0,0.599184,0.038249,0.004894,0.0
442,LegsSitting0,0.468179,0.139635,-0.354116,0.999992,0.481202,0.069902,-0.347386,0.999986,0.497640,0.058576,-0.347789,0.999989,0.512906,0.049339,-0.348243,0.999990,0.436048,0.107802,-0.302035,0.999979,0.423628,0.121495,-0.302157,0.999978,0.412402,0.134077,-0.302002,0.999959,0.559895,0.063187,-0.229076,0.999991,0.427374,0.161958,-0.026563,0.999944,0.525316,0.172137,-0.305431,...,0.461464,0.169094,-0.031812,0.0,0.463704,0.163542,-0.032293,0.0,0.476557,0.162138,-0.018959,0.0,0.459928,0.170983,-0.033360,0.0,0.461427,0.172776,-0.025352,0.0,0.459450,0.092892,0.001574,0.0,0.455362,0.098564,-0.000984,0.0,0.452307,0.102597,-0.004750,0.0,0.492963,0.062820,-0.003295,0.0,0.494657,0.051997,-0.003841,0.0
443,LegsSitting0,0.446794,0.151149,-0.325759,0.999978,0.456281,0.079720,-0.319048,0.999967,0.472009,0.066473,-0.319447,0.999970,0.486764,0.055044,-0.319886,0.999976,0.415035,0.121099,-0.272404,0.999952,0.403816,0.135058,-0.272526,0.999943,0.393374,0.147745,-0.272381,0.999912,0.531799,0.062904,-0.197807,0.999974,0.410554,0.172817,0.013026,0.999864,0.504772,0.179193,-0.274597,...,0.448294,0.173074,-0.032684,0.0,0.450197,0.167197,-0.033193,0.0,0.464234,0.164942,-0.020336,0.0,0.446698,0.174995,-0.034258,0.0,0.448704,0.176894,-0.026423,0.0,0.443577,0.096749,0.003173,0.0,0.439608,0.102797,0.000615,0.0,0.436431,0.107082,-0.003139,0.0,0.475935,0.062919,-0.002523,0.0,0.477006,0.051078,-0.003036,0.0
444,LegsSitting0,0.424962,0.162070,-0.376843,0.999602,0.426614,0.074983,-0.374538,0.999737,0.442361,0.061639,-0.374972,0.999509,0.456533,0.051597,-0.375338,0.999760,0.387294,0.124417,-0.326279,0.999797,0.376392,0.138976,-0.326452,0.999638,0.366389,0.151368,-0.326568,0.999748,0.504654,0.048747,-0.255001,0.999636,0.395915,0.165220,-0.032079,0.999700,0.490868,0.180553,-0.331060,...,0.430221,0.182708,-0.033662,0.0,0.431697,0.176687,-0.034604,0.0,0.445387,0.170800,-0.020541,0.0,0.428829,0.185243,-0.035196,0.0,0.430914,0.186062,-0.026621,0.0,0.418230,0.101620,-0.000904,0.0,0.414889,0.108940,-0.003239,0.0,0.412336,0.114407,-0.007029,0.0,0.448249,0.061542,-0.008552,0.0,0.448998,0.051143,-0.009432,0.0


In [ ]:
df[df['class']=='FacialAppearanceScore0']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,z3,v3,x4,y4,z4,v4,x5,y5,z5,v5,x6,y6,z6,v6,x7,y7,z7,v7,x8,y8,z8,v8,x9,y9,z9,v9,x10,y10,z10,...,x492,y492,z492,v492,x493,y493,z493,v493,x494,y494,z494,v494,x495,y495,z495,v495,x496,y496,z496,v496,x497,y497,z497,v497,x498,y498,z498,v498,x499,y499,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
65,FacialAppearanceScore0,0.522968,0.526882,-1.855416,0.999701,0.580803,0.416381,-1.785902,0.999715,0.617696,0.416476,-1.785318,0.999601,0.644648,0.418531,-1.786789,0.999675,0.474734,0.423107,-1.773776,0.999686,0.440045,0.426470,-1.775367,0.999577,0.406389,0.431959,-1.776155,0.999635,0.717270,0.490870,-1.174582,0.999598,0.369998,0.489074,-1.124440,0.999453,0.601065,0.661636,-1.610278,...,0.545574,0.555648,-0.076412,0.0,0.554847,0.546120,-0.075045,0.0,0.575788,0.566788,-0.038803,0.0,0.542350,0.557691,-0.080750,0.0,0.541772,0.564902,-0.060976,0.0,0.576216,0.405530,0.013579,0.0,0.565181,0.411873,0.003801,0.0,0.558391,0.415710,-0.007745,0.0,0.659523,0.390144,0.020235,0.0,0.668646,0.374419,0.020889,0.0
66,FacialAppearanceScore0,0.529291,0.529373,-2.114822,0.999628,0.585733,0.416531,-2.013008,0.999705,0.622352,0.416558,-2.013256,0.999548,0.651831,0.418770,-2.014415,0.999657,0.477708,0.423167,-2.002117,0.999678,0.443758,0.426660,-2.003317,0.999538,0.409230,0.432398,-2.005120,0.999628,0.722521,0.489811,-1.225815,0.999570,0.373964,0.493233,-1.180241,0.999423,0.608373,0.661673,-1.798866,...,0.574805,0.568820,-0.076992,0.0,0.583930,0.559446,-0.075013,0.0,0.604456,0.579027,-0.039860,0.0,0.571476,0.570671,-0.081447,0.0,0.570645,0.577925,-0.062376,0.0,0.601572,0.418932,0.020066,0.0,0.591037,0.424420,0.009928,0.0,0.584630,0.427608,-0.001875,0.0,0.683589,0.405316,0.028506,0.0,0.692822,0.387756,0.029539,0.0
67,FacialAppearanceScore0,0.540548,0.529652,-2.249519,0.999622,0.593439,0.416033,-2.120406,0.999708,0.629961,0.415995,-2.120878,0.999549,0.659369,0.418306,-2.121859,0.999657,0.486386,0.422731,-2.125642,0.999687,0.452113,0.426575,-2.126683,0.999547,0.417103,0.432400,-2.128441,0.999637,0.728536,0.485951,-1.228570,0.999576,0.380095,0.494170,-1.242893,0.999448,0.618440,0.661436,-1.893439,...,0.586208,0.573442,-0.076554,0.0,0.595279,0.563798,-0.074404,0.0,0.615472,0.582702,-0.038394,0.0,0.583035,0.575589,-0.081091,0.0,0.582123,0.582700,-0.061837,0.0,0.610276,0.418669,0.019512,0.0,0.599838,0.424249,0.009028,0.0,0.593751,0.427555,-0.003022,0.0,0.691853,0.403496,0.030116,0.0,0.700924,0.387032,0.031272,0.0
68,FacialAppearanceScore0,0.568931,0.534195,-2.203377,0.999580,0.614348,0.416032,-2.086279,0.999683,0.647616,0.416023,-2.086484,0.999500,0.676664,0.418319,-2.087412,0.999611,0.509513,0.422741,-2.097584,0.999676,0.471252,0.426659,-2.098725,0.999530,0.437225,0.432615,-2.100560,0.999630,0.740753,0.485282,-1.225426,0.999532,0.391850,0.496682,-1.270358,0.999448,0.636438,0.664341,-1.862010,...,0.608019,0.585419,-0.076865,0.0,0.617215,0.575954,-0.074482,0.0,0.635828,0.592502,-0.037025,0.0,0.605029,0.587699,-0.081507,0.0,0.603464,0.594215,-0.061836,0.0,0.626485,0.427438,0.019413,0.0,0.616497,0.433276,0.008374,0.0,0.611092,0.437000,-0.004147,0.0,0.706272,0.416497,0.032634,0.0,0.715139,0.398915,0.034196,0.0
69,FacialAppearanceScore0,0.583507,0.543396,-2.214478,0.999493,0.627153,0.417137,-2.103061,0.999644,0.658487,0.417013,-2.103203,0.999407,0.688331,0.418933,-2.104249,0.999554,0.521219,0.425516,-2.117365,0.999647,0.481278,0.430370,-2.118590,0.999473,0.447316,0.437164,-2.120596,0.999605,0.749505,0.485191,-1.245664,0.999446,0.397162,0.503744,-1.316264,0.999388,0.648077,0.669716,-1.873640,...,0.607574,0.587234,-0.076528,0.0,0.616787,0.577601,-0.074419,0.0,0.635344,0.595809,-0.036567,0.0,0.604604,0.589568,-0.081149,0.0,0.603153,0.596224,-0.061226,0.0,0.629510,0.427869,0.016588,0.0,0.619243,0.433693,0.005823,0.0,0.613551,0.437430,-0.006240,0.0,0.711623,0.413617,0.029507,0.0,0.720616,0.397049,0.030820,0.0
70,FacialAppearanceScore0,0.582490,0.543052,-2.150174,0.999430,0.626720,0.415985,-2.045846,0.999630,0.659156,0.416171,-2.045842,0.9993

In [ ]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
y_test

424              LegsSitting0
29     FacialAppearanceScore2
169         HeadSittingScore1
131         HeadSittingScore1
390              LegsSitting1
                ...          
148         HeadSittingScore1
297                 HandForm1
101         HeadSittingScore3
202         HeadSittingScore1
375                 HandForm0
Name: class, Length: 134, dtype: object

### **3.2 Train Machine Learning Classification Model**

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:

fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
fit_models

{'gb': Pipeline(memory=None,
          steps=[('standardscaler',
                  StandardScaler(copy=True, with_mean=True, with_std=True)),
                 ('gradientboostingclassifier',
                  GradientBoostingClassifier(ccp_alpha=0.0,
                                             criterion='friedman_mse', init=None,
                                             learning_rate=0.1, loss='deviance',
                                             max_depth=3, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100,
                                 

In [ ]:
fit_models['rc'].predict(X_test)

array(['LegsSitting0', 'FacialAppearanceScore2', 'HeadSittingScore1',
       'HeadSittingScore1', 'LegsSitting1', 'HeadSittingScore3',
       'FacialAppearanceScore2', 'HandForm3', 'FacialAppearanceScore2',
       'FacialAppearanceScore0', 'HeadSittingScore1',
       'FacialAppearanceScore0', 'HeadSittingScore1', 'HeadSittingScore1',
       'HandForm1', 'FacialAppearanceScore0', 'FacialAppearanceScore3',
       'HeadSittingScore1', 'HeadSittingScore1', 'FacialAppearanceScore2',
       'FacialAppearanceScore3', 'LegsSitting0', 'HandForm0',
       'HeadSittingScore1', 'LegsSitting0', 'LegsSitting3',
       'HeadSittingScore1', 'HeadSittingScore1', 'HeadSittingScore1',
       'FacialAppearanceScore2', 'HeadSittingScore1', 'LegsSitting1',
       'HandForm0', 'HandForm0', 'HeadSittingScore1', 'HandForm3',
       'HandForm0', 'HeadSittingScore1', 'LegsSitting0', 'LegsSitting1',
       'HandForm1', 'LegsSitting0', 'HandForm3', 'HeadSittingScore1',
       'TrunkSittingScore1', 'HeadSittingScor

### **3.3 Evaluate and Serialize Model¶**

In [ ]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))


lr 0.917910447761194
rc 0.9328358208955224
rf 0.9253731343283582
gb 0.8955223880597015


In [ ]:
fit_models['rf'].predict(X_test)

array(['LegsSitting0', 'FacialAppearanceScore2', 'HeadSittingScore1',
       'HeadSittingScore1', 'LegsSitting1', 'HeadSittingScore3',
       'FacialAppearanceScore2', 'HandForm3', 'FacialAppearanceScore2',
       'FacialAppearanceScore0', 'HeadSittingScore1',
       'FacialAppearanceScore0', 'HeadSittingScore1', 'HeadSittingScore0',
       'HandForm1', 'FacialAppearanceScore0', 'FacialAppearanceScore3',
       'HeadSittingScore1', 'HeadSittingScore1', 'FacialAppearanceScore2',
       'FacialAppearanceScore3', 'LegsSitting0', 'HandForm0',
       'HeadSittingScore1', 'LegsSitting0', 'LegsSitting3',
       'HeadSittingScore1', 'HeadSittingScore1', 'HeadSittingScore1',
       'FacialAppearanceScore2', 'HeadSittingScore1', 'LegsSitting1',
       'HandForm0', 'HandForm0', 'HeadSittingScore1', 'HandForm3',
       'HandForm0', 'HeadSittingScore1', 'LegsSitting0',
       'TrunkSittingScore1', 'HandForm1', 'LegsSitting0', 'HandForm3',
       'HeadSittingScore1', 'TrunkSittingScore1', 'HeadSitti

In [ ]:
y_test

424              LegsSitting0
29     FacialAppearanceScore2
169         HeadSittingScore1
131         HeadSittingScore1
390              LegsSitting1
                ...          
148         HeadSittingScore1
297                 HandForm1
101         HeadSittingScore3
202         HeadSittingScore1
375                 HandForm0
Name: class, Length: 134, dtype: object

In [ ]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## **4. Make Detections with Model**

In [ ]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
model

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=F

In [ ]:
# start streaming video from webcam
video_stream()

# # label for video
label_html = 'Capturing...' 
# initialze bounding box to empty
bbox = ''
count = 0 
# image = '' 

#Intiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while True:
      js_reply = video_frame(label_html, bbox )
    
      if not js_reply:
       break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      #Recolour feed
      image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      #Make Detections
      results = holistic.process(image)
      print(results.face_landmarks)

      #face_Landmarks, pose_Landmarks, left_Landmarks, right_Landmarks

      #Recolor image back ot BGR for Rendering 
      image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

      #Draw Face landmarks 
      mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)

      #Right hand 
      mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      #Left hand
      mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      # Pose Detections
      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
       
      # Export coordinates
      try:
          # Extract Pose landmarks
          pose = results.pose_landmarks.landmark
          pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
          
          # Extract Face landmarks
          face = results.face_landmarks.landmark
          face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
          
          # Concate rows
          row = pose_row+face_row
          
#             # Append class name 
#             row.insert(0, class_name)
          
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

          # Make Detections
          X = pd.DataFrame([row])
          body_language_class = model.predict(X)[0]
          body_language_prob = model.predict_proba(X)[0]
          print(body_language_class, body_language_prob)
          
          # Grab ear coords
          coords = tuple(np.multiply(
                          np.array(
                              (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                      , [640,480]).astype(int))
          
          cv2.rectangle(image, 
                        (coords[0], coords[1]+5), 
                        (coords[0]+len(body_language_class)*20, coords[1]-30), 
                        (245, 117, 16), -1)
          cv2.putText(image, body_language_class, coords, 
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          
          # Get status box
          cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
          
          # Display Class
          cv2.putText(image, 'CLASS'
                      , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
          cv2.putText(image, body_language_class.split(' ')[0]
                      , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          
          # Display Probability
          cv2.putText(image, 'PROB'
                      , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
          cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                      , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          
      except:
          pass
                        
    

           

    
      cv2_imshow(image)

In [ ]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))